In [5]:
%matplotlib inline

import shutil
import os
from joblib import dump, load
import urllib3
import certifi
from Bio import SeqIO
import Bio
from glob import glob
import json
from IPython import display
import pandas as pd
import numpy as np
import networkx as nx
from collections import OrderedDict

import matplotlib.pyplot as plt
import seaborn as sns
import re

In [27]:
# HIV properties
patients = ["p{}".format(i) for i in range(1,12)]
hiv_regions = ["V3", "PR", "psi", "vpr", "vpu", "p1", "p2", "p6", "p7", "p15", "p17", "RRE"]

In [10]:
def download_hivevo_references(folder):
    '''
    Downloading references for each patient and storing them in dir folder/references
    '''
    global patients    
    
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                               ca_certs=certifi.where())
    
    if not os.path.isdir(folder):
        os.mkdir(folder)
    
    for patient in patients:
        api = "https://hiv.biozentrum.unibas.ch/api/data/referenceSequence"
        url = "/".join((api, patient))
        file_name = folder + "_".join(("reference", patient)) + ".fasta"
        
        with http.request('GET', url, preload_content=False) as res, open(file_name, 'wb') as out_file:
            shutil.copyfileobj(res, out_file)


folder = "../data/references/"
download_hivevo_references(folder)

In [50]:
def extracting_region_from_reference(region, reference_path, folder, pat):
    '''
    Extracting special region from reference
    '''
    
    if not os.path.isdir(folder + pat + '/' + region):
        os.mkdir(folder + pat + '/' + region)
    
    with open(reference_path) as f:
        reference_info = json.load(f)
    for reg in reference_info['features']:
        if reg['name'] == region:
            loc = reg['location'][0]
            break
        else:
            continue
    patient = pat
    print(patient)
    sequence = reference_info['seq']
    print(sequence)
    res = r'/'+re.search(r'[\w]*\.fasta', reference_path).[0].replace(patient, "_".join((patient, region)))
    
    with open(folder+region+res, 'w') as write_file:
        write_file.write('>'+reference_info['description'].replace('genomewide', 'region='+region)+'\n')
        write_file.write(sequence)

        

# Need repair  
#folder = '../data/references/'
#for patient in patients:
#    for region in hiv_regions:
#        ref_path = '../data/references/reference_' + patient + '.fasta'
#        extracting_region_from_reference(region, ref_path, folder, patient)

OSError: [Errno 2] No such file or directory: '../data/references/p2/V3'

In [75]:
def json2fasta(folder, path_json):
    
    with open(path, "w") as fasta_file:
        for obj in json_file:
            line_1 = '>' + obj['name'] + '\n'
            line_2 = obj['sequence'] + '\n'
            lines = [line_1, line_2]
            fasta_file.writelines(lines)
            
json2fasta('../data/regions/', '/home/kharlamov/Документы/Project_HIV/data/regions/p1_V3.fasta')

IOError: [Errno 2] No such file or directory: '/home/kharlamov/\xd0\x94\xd0\xbe\xd0\xba\xd1\x83\xd0\xbc\xd0\xb5\xd0\xbd\xd1\x82\xd1\x8b/Project_HIV/data/fasta/regions/p1_V3.fasta'

In [13]:
def add_ref_reg2fasta(path_fasta, path_ref_region):
    '''
    Adding exact region from reference file to fasta with haplotypes
    '''
    with open(path_fasta, 'a') as fasta, open(path_ref_region) as ref:
        for line in ref:
            #print(line)
            fasta.write(line)
            
#add_ref_reg2fasta('data/fasta/hivevo_p4_V3.fasta', 'data/references/V3/hivevo_reference_p4_V3.fasta')